In [66]:
train.data <- read.csv(file.path("..", "data", "training_data.csv"))
test.data <- read.csv(file.path("..", "data", "test_data.csv"))
sampleSubmission <- read.csv(file.path("..", "data", "sampleSubmission.csv"))

In [67]:
#delets dependent variables
train.dub <- train.data[!duplicated(as.list(train.data))]


In [26]:
install.packages("tidymodels")

also installing the dependencies ‘infer’, ‘tune’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [91]:
library(tidymodels)
library(leaps)

#data set with VALENCE.PLEASANTNESS + V.. (with low varinace)
length <- length(train.dub[1,])
idx.low.var <- apply(train.dub[,4:length], 2, var) < 0.24 # identify pixels with low variance 
V.data <- data.frame(train.dub[,4:length])
V.data <- data.frame(V.data[,!idx.low.var])
V.data$VALENCE.PLEASANTNESS <- train.data$VALENCE.PLEASANTNESS

In [92]:
predict.regsubsets <- function(object, newdata, id, form = as.formula(object$call[[2]])) {
    mat = model.matrix(form, newdata)
    coefi = coef(object, id=id)
    xvars = names(coefi)
    mat[,xvars]%*%coefi
}

In [94]:
#Forward selection with K-fold Cross Validation 
nvmax = 100
validation_data <- vfold_cv(V.data, v = 5)

fit_and_evaluate <- function(fold, formula = VALENCE.PLEASANTNESS ~ .) {
    fit <- regsubsets(formula, analysis(fold), nvmax = nvmax, method = 'forward')
    valid.set <- assessment(fold)
    sapply(seq(1, fit$nvmax - 1),
           function(id) mean((valid.set$VALENCE.PLEASANTNESS - predict.regsubsets(fit, valid.set, id, formula))^2))
}
           
cv.errors <- sapply(validation_data$splits, fit_and_evaluate)
rowMeans(cv.errors)
          

[1] 527.1659 538.2637 541.0899 542.8208 540.0992 546.6489 549.0874 546.0457
 [9] 539.4517 539.8563 540.7017 543.2733 540.6052 540.4476 540.2542 542.7808
[17] 539.7984 541.2203 536.7433 535.9744 533.0781 533.9055 531.6613 535.4693
[25] 539.2929 538.3711 537.2966 537.5876 537.1391 541.1021 543.7692 541.4816
[33] 543.5126 545.2698 544.4663 543.4939 543.0612 544.6368 546.9985 547.7966
[41] 547.4482 547.7126 547.8262 547.0542 548.3255 547.4395 550.4920 549.8940
[49] 549.7469 548.8752 548.8183 547.0073 546.7074 549.1274 551.9418 552.9541
[57] 552.7700 550.3493 549.4324 548.4850 548.7444 547.8579 546.5691 546.4523
[65] 545.9825 546.1270 546.2335 547.0680 548.2250 548.0277 547.9844 548.1175
[73] 548.3965 548.0253 547.7657 547.8853 547.6782 547.1512 546.8262 546.7847
[81] 546.9641 547.0106 547.0281